In [1]:
%pip install iterative-stratification


Note: you may need to restart the kernel to use updated packages.


In [2]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from tensorflow.keras import mixed_precision 
import os
import gc
import ctypes

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from tensorflow.keras.layers import (Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Flatten, Dense,GlobalAveragePooling1D)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,
                                      CSVLogger, EarlyStopping)
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from collections import Counter
from typing import Tuple, Optional, Callable
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.losses import KLDivergence
from tensorflow.keras.utils import Sequence
import pandas as pd 



2024-02-23 21:13:47.791943: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 21:13:47.792051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 21:13:47.948811: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# GPU set up

In [4]:

print('tensorflow version:',tf.__version__)

# CUDA 0,1
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# gpu strategy
gpus = tf.config.list_physical_devices('GPU')
if len(gpus) <= 1:
    strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')
    print(f'Using {len(gpus)} gpus')
else: 
    strategy = tf.distribute.MirroredStrategy()
    print(f'Using {len(gpus)} gpus')
    
# warning filtering
import warnings
warnings.filterwarnings('ignore')


gpu_memory_fraction = 0.8
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))


policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)



tensorflow version: 2.15.0
Using 2 gpus


# Preprocessing and SET UP

In [5]:
train_csv = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")
print('train shape: ',train_csv.shape)
display(train_csv.head())
TARGETS = train_csv.columns[-6:]
TARGETS

train shape:  (106800, 15)


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


Index(['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote',
       'other_vote'],
      dtype='object')

In [6]:
train = train_csv.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']

tmp = train_csv.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})

train['max'] = tmp

tmp = train_csv.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = train_csv.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = train_csv.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()

#save the train data
# train.to_csv('data_nonoverlap.csv',index=False)

Train non-overlapp eeg_id shape: (17089, 12)


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


In [7]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.utils import indexable, _safe_indexing
from sklearn.utils.validation import _num_samples
from sklearn.model_selection._split import _validate_shuffle_split
from itertools import chain
from sklearn.model_selection import train_test_split



def multilabel_train_test_split(*arrays,
                                test_size=None,
                                train_size=None,
                                random_state=None,
                                shuffle=True,
                                stratify=None):
    """
    Train test split for multilabel classification. Uses the algorithm from: 
    'Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of Multi-Label Data'.
    """
    if stratify is None:
        return train_test_split(*arrays, test_size=test_size,train_size=train_size,
                                random_state=random_state, stratify=None, shuffle=shuffle)
    
    assert shuffle, "Stratified train/test split is not implemented for shuffle=False"
    
    n_arrays = len(arrays)
    arrays = indexable(*arrays)
    n_samples = _num_samples(arrays[0])
    n_train, n_test = _validate_shuffle_split(
        n_samples, test_size, train_size, default_test_size=0.25
    )
    cv = MultilabelStratifiedShuffleSplit(test_size=n_test, train_size=n_train, random_state=123)
    train, test = next(cv.split(X=arrays[0], y=stratify))

    return list(
        chain.from_iterable(
            (_safe_indexing(a, train), _safe_indexing(a, test)) for a in arrays
        )
    )
    
    


def create_split(data_,label_col):
    # Assuming your dataset is stored in a DataFrame called 'df'

    # Split the dataset without duplicates
    train_data_no_duplicates, temp_data_no_duplicates = multilabel_train_test_split(data_, stratify=data_[label_col], test_size=0.3, random_state=42)
    val_data_no_duplicates, test_data_no_duplicates = multilabel_train_test_split(temp_data_no_duplicates, stratify=temp_data_no_duplicates[TARGETS] , test_size=0.66, random_state=42)

    # Merge back the duplicates with the subsets
  
    # Merge back the special cases with the train set
    

    return train_data_no_duplicates, val_data_no_duplicates, test_data_no_duplicates

In [8]:
train, val, test = create_split(train,TARGETS)


In [9]:
import concurrent.futures

def eeg_tonumpy(df):
    eeg_ids = df['eeg_id'].values.tolist()

    directory ='/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'


    eegs = pd.DataFrame(eeg_ids, columns=['eeg_id'])
    eegs['file_path'] = eegs['eeg_id'].apply(lambda x: os.path.join(directory, f"{x}.parquet"))
    eggs_path = eegs['file_path'].values

    eeg_numpy = []

    def read_eeg_data(file):
        eeg_data = pd.read_parquet(file)
        # Convert to numpy
        eeg_data = eeg_data.to_numpy()
        rows = len(eeg_data)
        offset = (rows-10_000)//2
        eeg = eeg_data[offset:offset+10_000]
        
        return eeg

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(read_eeg_data, file) for file in eggs_path]
        for future in concurrent.futures.as_completed(futures):
            eeg_data = future.result()
            eeg_numpy.append(eeg_data)
                
        
    return  np.array(eeg_numpy)
            
            

In [10]:
test_eeg= eeg_tonumpy(test)
train_eeg=eeg_tonumpy(train)
val_eeg=eeg_tonumpy(val)


In [11]:
#EEG with all Zeros

from tqdm import tqdm

def get_badEEG(X_data):
    channel_zeros_count = {f"Channel {i+1}": [] for i in range(X_data.shape[-1])}
    unique_indices = set()

    # Display "analyzing" before starting the analysis
    print("Analyzing...")

    # Use tqdm for progress display
    for sample_index, sample in tqdm(enumerate(X_data), desc="Samples", dynamic_ncols=True, leave=True):
        for i, channel in enumerate(sample.T, start=1):
            if np.all(np.isnan(channel) | (channel == 0)):
                channel_zeros_count[f"Channel {i}"].append(sample_index)
                unique_indices.add(sample_index)

    # Display "Finished" in highlighted green
    print("\033[92mFinished\033[0m")

    # Print the report while iterating
    for channel, indices in channel_zeros_count.items():
        print(f"{channel}: {len(indices)} samples have all zeros. Indices: {indices}")

    # Create a mask for filtering
    mask = np.ones(len(X_data), dtype=bool)
    mask[list(unique_indices)] = False
    X_val_filtered = X_data[mask]
    
    return X_val_filtered, list(unique_indices)

In [12]:
train_filtered,train_filtered_index = get_badEEG(train_eeg)
val_filtered,val_filtered_index = get_badEEG(val_eeg)
test_filtered,test_filtered_index = get_badEEG(test_eeg)



Analyzing...


Samples: 11963it [00:09, 1204.78it/s]


Finished
Channel 1: 0 samples have all zeros. Indices: []
Channel 2: 0 samples have all zeros. Indices: []
Channel 3: 0 samples have all zeros. Indices: []
Channel 4: 0 samples have all zeros. Indices: []
Channel 5: 0 samples have all zeros. Indices: []
Channel 6: 0 samples have all zeros. Indices: []
Channel 7: 0 samples have all zeros. Indices: []
Channel 8: 0 samples have all zeros. Indices: []
Channel 9: 0 samples have all zeros. Indices: []
Channel 10: 0 samples have all zeros. Indices: []
Channel 11: 0 samples have all zeros. Indices: []
Channel 12: 0 samples have all zeros. Indices: []
Channel 13: 0 samples have all zeros. Indices: []
Channel 14: 0 samples have all zeros. Indices: []
Channel 15: 0 samples have all zeros. Indices: []
Channel 16: 0 samples have all zeros. Indices: []
Channel 17: 0 samples have all zeros. Indices: []
Channel 18: 0 samples have all zeros. Indices: []
Channel 19: 0 samples have all zeros. Indices: []
Channel 20: 54 samples have all zeros. Indices: [1

Samples: 1746it [00:01, 1201.66it/s]


Finished
Channel 1: 0 samples have all zeros. Indices: []
Channel 2: 0 samples have all zeros. Indices: []
Channel 3: 0 samples have all zeros. Indices: []
Channel 4: 0 samples have all zeros. Indices: []
Channel 5: 0 samples have all zeros. Indices: []
Channel 6: 0 samples have all zeros. Indices: []
Channel 7: 0 samples have all zeros. Indices: []
Channel 8: 0 samples have all zeros. Indices: []
Channel 9: 0 samples have all zeros. Indices: []
Channel 10: 0 samples have all zeros. Indices: []
Channel 11: 0 samples have all zeros. Indices: []
Channel 12: 0 samples have all zeros. Indices: []
Channel 13: 0 samples have all zeros. Indices: []
Channel 14: 0 samples have all zeros. Indices: []
Channel 15: 0 samples have all zeros. Indices: []
Channel 16: 0 samples have all zeros. Indices: []
Channel 17: 0 samples have all zeros. Indices: []
Channel 18: 0 samples have all zeros. Indices: []
Channel 19: 0 samples have all zeros. Indices: []
Channel 20: 9 samples have all zeros. Indices: [32

Samples: 3380it [00:02, 1231.69it/s]


Finished
Channel 1: 0 samples have all zeros. Indices: []
Channel 2: 0 samples have all zeros. Indices: []
Channel 3: 0 samples have all zeros. Indices: []
Channel 4: 0 samples have all zeros. Indices: []
Channel 5: 0 samples have all zeros. Indices: []
Channel 6: 0 samples have all zeros. Indices: []
Channel 7: 0 samples have all zeros. Indices: []
Channel 8: 0 samples have all zeros. Indices: []
Channel 9: 0 samples have all zeros. Indices: []
Channel 10: 0 samples have all zeros. Indices: []
Channel 11: 0 samples have all zeros. Indices: []
Channel 12: 0 samples have all zeros. Indices: []
Channel 13: 0 samples have all zeros. Indices: []
Channel 14: 0 samples have all zeros. Indices: []
Channel 15: 0 samples have all zeros. Indices: []
Channel 16: 0 samples have all zeros. Indices: []
Channel 17: 0 samples have all zeros. Indices: []
Channel 18: 0 samples have all zeros. Indices: []
Channel 19: 0 samples have all zeros. Indices: []
Channel 20: 15 samples have all zeros. Indices: [2

In [13]:
del train_eeg
del val_eeg
del test_eeg
gc.collect()

0

In [14]:
#Filter to remove all bad EEG values
train_data_filtered = train.drop(train.index[train_filtered_index])
val_data_filtered = val.drop(val.index[val_filtered_index])
test_data_filtered = test.drop(test.index[test_filtered_index])
Y_train = train_data_filtered.iloc[:, 5:11]
Y_val=val_data_filtered.iloc[:, 5:11]
Y_test=test_data_filtered.iloc[:, 5:11]

In [15]:
#Normalizing values and removing NAN 
from sklearn.preprocessing import StandardScaler  #scaler   
def getParam(X):
    #to float32
    a, b, c = X.shape
    
    ax = np.reshape(X, (a * b * c))
    ax = np.reshape(X, (-1, 1))
    print(ax.shape)
    # scaler = StandardScaler()
    mean=np.nanmean(ax)
    std=np.nanstd(ax)
    
    # scaler.fit(ax)
    # mean = scaler.mean_
    # std = np.sqrt(scaler.var_)
    return mean,std


def standardize(X, mean, std):
    return (X - mean) / std


mean,std=getParam(train_filtered)
print(mean,std)

train_normalized = standardize(train_filtered, mean, std)
del train_filtered
val_normalized = standardize(val_filtered, mean, std)
del val_filtered
test_normalized = standardize(test_filtered, mean, std)
del test_filtered



(2381800000, 1)
79.486145 5467.902


In [16]:
train_normalized[np.isnan(train_normalized)] =mean
val_normalized[np.isnan(val_normalized)] = mean
test_normalized[np.isnan(test_normalized)] = mean

In [17]:
X_train=train_normalized
X_val=val_normalized
X_test=test_normalized


#  Resnet18 Model

In [18]:

from tensorflow.keras.layers import (Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Flatten, Dense,GlobalAveragePooling1D)
from tensorflow.keras.models import Model
import numpy as np
import os



class ResidualUnit(object):
    """Residual unit block (unidimensional).
    Parameters
    ----------
    n_samples_out: int
        Number of output samples.
    n_filters_out: int
        Number of output filters.
    kernel_initializer: str, optional
        Initializer for the weights matrices. See Keras initializers. By default it uses
        'he_normal'.
    dropout_keep_prob: float [0, 1), optional
        Dropout rate used in all Dropout layers. Default is 0.8
    kernel_size: int, optional
        Kernel size for convolutional layers. Default is 17.
    preactivation: bool, optional
        When preactivation is true use full preactivation architecture proposed
        in [1]. Otherwise, use architecture proposed in the original ResNet
        paper [2]. By default it is true.
    postactivation_bn: bool, optional
        Defines if you use batch normalization before or after the activation layer (there
        seems to be some advantages in some cases:
        https://github.com/ducha-aiki/caffenet-benchmark/blob/master/batchnorm.md).
        If true, the batch normalization is used before the activation
        function, otherwise the activation comes first, as it is usually done.
        By default it is false.
    activation_function: string, optional
        Keras activation function to be used. By default 'relu'.
    References
    ----------
    .. [1] K. He, X. Zhang, S. Ren, and J. Sun, "Identity Mappings in Deep Residual Networks,"
           arXiv:1603.05027 [cs], Mar. 2016. https://arxiv.org/pdf/1603.05027.pdf.
    .. [2] K. He, X. Zhang, S. Ren, and J. Sun, "Deep Residual Learning for Image Recognition," in 2016 IEEE Conference
           on Computer Vision and Pattern Recognition (CVPR), 2016, pp. 770-778. https://arxiv.org/pdf/1512.03385.pdf
    """

    def __init__(self, n_samples_out, n_filters_out, kernel_initializer='he_normal',
                 dropout_keep_prob=0.8, kernel_size=17, preactivation=True,
                 postactivation_bn=False, activation_function='relu'):
        self.n_samples_out = n_samples_out
        self.n_filters_out = n_filters_out
        self.kernel_initializer = kernel_initializer
        self.dropout_rate = 1 - dropout_keep_prob
        self.kernel_size = kernel_size
        self.preactivation = preactivation
        self.postactivation_bn = postactivation_bn
        self.activation_function = activation_function

    def _skip_connection(self, y, downsample, n_filters_in):
        """Implement skip connection."""
        # Deal with downsampling
        if downsample > 1:
            y = MaxPooling1D(downsample, strides=downsample, padding='same')(y)
        elif downsample == 1:
            y = y
        else:
            raise ValueError("Number of samples should always decrease.")
        # Deal with n_filters dimension increase
        if n_filters_in != self.n_filters_out:
            # This is one of the two alternatives presented in ResNet paper
            # Other option is to just fill the matrix with zeros.
            y = Conv1D(self.n_filters_out, 1, padding='same',
                       use_bias=False, kernel_initializer=self.kernel_initializer)(y)
        return y

    def _batch_norm_plus_activation(self, x):
        if self.postactivation_bn:
            x = Activation(self.activation_function)(x)
            x = BatchNormalization(center=False, scale=False)(x)
        else:
            x = BatchNormalization()(x)
            x = Activation(self.activation_function)(x)
        return x

    def __call__(self, inputs):
        """Residual unit."""
        x, y = inputs
        n_samples_in = y.shape[1]
        downsample = n_samples_in // self.n_samples_out
        n_filters_in = y.shape[2]
        y = self._skip_connection(y, downsample, n_filters_in)
        # 1st layer
        x = Conv1D(self.n_filters_out, self.kernel_size, padding='same',
                   use_bias=False, kernel_initializer=self.kernel_initializer)(x)
        x = self._batch_norm_plus_activation(x)
        if self.dropout_rate > 0:
            x = Dropout(self.dropout_rate)(x)

        # 2nd layer
        x = Conv1D(self.n_filters_out, self.kernel_size, strides=downsample,
                   padding='same', use_bias=False,
                   kernel_initializer=self.kernel_initializer)(x)
        if self.preactivation:
            x = Add()([x, y])  # Sum skip connection and main connection
            y = x
            x = self._batch_norm_plus_activation(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
        else:
            x = BatchNormalization()(x)
            x = Add()([x, y])  # Sum skip connection and main connection
            x = Activation(self.activation_function)(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
            y = x
        return [x, y]


def get_model2(n_classes, last_layer='softmax'): #try with softmax 'sigmoid' initially
    kernel_size = 16 
    kernel_initializer = 'he_normal'
    signal = Input(shape=(10000, 20), dtype=np.float32, name='signal') #change to my input size 2500,12
 #change to my input size 2500,12
    x = signal
    x = Conv1D(64, kernel_size, padding='same', use_bias=False,
               kernel_initializer=kernel_initializer)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x, y = ResidualUnit(1024, 128, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, x])
    x, y = ResidualUnit(256, 196, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x, y = ResidualUnit(64, 256, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x, _ = ResidualUnit(16, 320, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    # x=GlobalAveragePooling1D()(x)
    x = Flatten()(x)
    diagn = Dense(n_classes, activation=last_layer, kernel_initializer=kernel_initializer, dtype='float32')(x)
    # diagn=GlobalAveragePooling1D()(x)

    model = Model(signal, diagn)
    return model


model = get_model2(6)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 signal (InputLayer)         [(None, 10000, 20)]          0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 10000, 64)            20480     ['signal[0][0]']              
                                                                                                  
 batch_normalization (Batch  (None, 10000, 64)            256       ['conv1d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 10000, 64)            0         ['batch_normalization[0][0

# Training 

# Helpers

In [19]:
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x.astype(np.float32), batch_y #makes sure output isin float32
    
    
class CleanMemory(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()

    
class ResetLR(tf.keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
    default_lr = 0.1
    previous_lr = self.model.optimizer.lr.read_value()
    if previous_lr!=0.01:
      print("Resetting learning rate from {} to {}".format(previous_lr, default_lr))
      self.model.optimizer.lr.assign(default_lr)

class ReduceLRBacktrack(ReduceLROnPlateau):
    def __init__(self, best_path, *args, **kwargs):
        super(ReduceLRBacktrack, self).__init__(*args, **kwargs)
        self.best_path = best_path

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            logging.warning('Reduce LR on plateau conditioned on metric `%s` '
                            'which is not available. Available metrics are: %s',
                             self.monitor, ','.join(list(logs.keys())))
        if not self.monitor_op(current, self.best): # not new best
            if not self.in_cooldown(): # and we're not in cooldown
                if self.wait+1 >= self.patience: # going to reduce lr
                    # load best model so far
                    print("Backtracking to best model before reducting LR")
                    self.model.load_weights(self.best_path)

        super().on_epoch_end(epoch, logs) # actually reduce LR

In [20]:
config= {'epochs':50, 'learning_rate':0.01, 'batch_size': 256}
out_dir='/kaggle/working'
loss = tf.keras.losses.KLDivergence()
condition_name='V1'

lr = config['learning_rate']
batch_size =config['batch_size']
opt = Adam(lr)
opt = mixed_precision.LossScaleOptimizer(opt)

callbacks = [ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=7,
                               min_lr=lr / 100),
             EarlyStopping(patience=9,  # Patience should be larger than the one in ReduceLROnPlateau
                           min_delta=0.00001)]

model = get_model2(6)
model.compile(loss=loss, optimizer=opt, metrics=['accuracy', #remplacer par tes metriques
                                                                tf.keras.metrics.CategoricalAccuracy(name='acc'),
                                                                #  tf.keras.metrics.AUC(curve='ROC', name='ROC'),
                                                                #  tf.keras.metrics.AUC(curve='PR', name='PR')
                                                                ], run_eagerly=False)

    

c1 = ModelCheckpoint(os.path.join(out_dir,'{}_benchmark/'.format(condition_name), 'best_model.h5'), 
                    save_best_only=False,
                    save_weights_on=True,
                    monitor='val_loss')
    
    

train_gen = DataGenerator(X_train, Y_train, config['batch_size']) #if don't like go to 512
val_gen = DataGenerator(X_val, Y_val,config['batch_size'])

callbacks +=[CleanMemory(),c1]
callbacks += [TensorBoard(log_dir='./logs', write_graph=False),
                 CSVLogger('training.log', append=False)]  # Change append to true if continuing training


    
history = model.fit(train_gen,
                    epochs=config['epochs'],
                    initial_epoch=0,  # If you are continuing a interrupted section change here
                    callbacks=callbacks,
                    validation_data=val_gen,
                    verbose=1)
# Save final result
del X_train
del Y_train
del X_val
del Y_val



# model.save_weights(os.path.join(out_dir,'{}_benchmark/'.format(condition_name), 'model.h5')) #useless if save at checkpoint


del model
# del early_stop
# del lrplateau
# tf.keras.clear_session()
tf.compat.v1.reset_default_graph()
gc.collect()





Epoch 1/50


I0000 00:00:1708723703.810270     104 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


47/47 [==============================] - 1051s 8s/step - loss: 1.3840 - accuracy: 0.4140 - acc: 0.4140 - val_loss: 1.4938 - val_accuracy: 0.4088 - val_acc: 0.4088 - lr: 0.0010
Epoch 2/50
47/47 [==============================] - 44s 928ms/step - loss: 1.3241 - accuracy: 0.4216 - acc: 0.4216 - val_loss: 1.3329 - val_accuracy: 0.4122 - val_acc: 0.4122 - lr: 0.0010
Epoch 3/50
47/47 [==============================] - 44s 929ms/step - loss: 1.3137 - accuracy: 0.4228 - acc: 0.4228 - val_loss: 1.2968 - val_accuracy: 0.4134 - val_acc: 0.4134 - lr: 0.0010
Epoch 4/50
47/47 [==============================] - 44s 928ms/step - loss: 1.3147 - accuracy: 0.4220 - acc: 0.4220 - val_loss: 1.3008 - val_accuracy: 0.4122 - val_acc: 0.4122 - lr: 0.0010
Epoch 5/50
47/47 [==============================] - 44s 928ms/step - loss: 1.3128 - accuracy: 0.4218 - acc: 0.4218 - val_loss: 1.3021 - val_accuracy: 0.4122 - val_acc: 0.4122 - lr: 0.0010
Epoch 6/50
47/47 [==============================] - 44s 925ms/step - los

17